Don't forget to create a `.env` file in this directory or somewhere above with `ES_USERNAME=<JPL_USERNAME>` and `ES_PASSWORD=<JPL_PASSWORD>`.

In [1]:
from dotenv import dotenv_values
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Q, Search
from tqdm import tqdm
import urllib3

Because HySDS doesn't have ssl certificates...

In [24]:
urllib3.disable_warnings()

In [3]:
config = dotenv_values()
ES_USERNAME = config['ES_USERNAME']
ES_PASSWORD = config['ES_PASSWORD']

In [4]:
GRQ_URL = 'https://100.104.62.10/grq_es/'
grq_client = Elasticsearch(GRQ_URL,
                           http_auth=(ES_USERNAME, ES_PASSWORD),
                           verify_certs=False,
                           read_timeout=50000,
                           terminate_after=2500,
                           )
search = Search(using=grq_client,
                # wildcard is where the version is
                index='grq_*_l3_dswx_hls')

/Users/cmarshak/mambaforge/envs/dswx_val/lib/python3.11/site-packages/elasticsearch/connection/http_urllib3.py:209: UserWarning: Connecting to https://100.104.62.10:443 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [5]:
if not grq_client.ping():
    raise ValueError('Either JPL username/password is wrong or not connected to VPN')

All the records in the index

In [6]:
total_dswx_hls = search.count()
total_dswx_hls

66813

# Figure out which SAS vesion you want

Go here: https://github.com/nasa/PROTEUS/releases

As of writing, latest is `v0.5.3` - ES removes the `v`.

In [7]:
CURRENT_DSW_HLS_VERSION='0.5.3'

In [8]:
q_qs = Q('query_string',
         query=f'\"{CURRENT_DSW_HLS_VERSION}\"',
         default_field="metadata.sas_version")

query = search.query(q_qs)
resp = query.execute()

In [9]:
len(resp.hits)

0

There are none. So, let's get the previous version.

In [10]:
PREVIOUS_DSWX_VERSION='0.5.2'

In [11]:
q_qs = Q('query_string',
         query=f'\"{PREVIOUS_DSWX_VERSION}\"',
         default_field="metadata.sas_version")

query = search.query(q_qs)
resp = query.execute()

In [12]:
query.count()

66813

# Get one

In [13]:
search_sub = search[:1]
resp = search_sub.execute()
hit = resp.hits[0]
hit

<Hit(grq_v1.0_l3_dswx_hls/OPERA_L3_DSWx-HLS_T41TPF_20221206T061737Z_20230212T000645Z_L8_30_v1.0): {'dataset_type': 'L3_DSWx_HLS', 'metadata': {'lineage': ['/d...}>

In [16]:
data_dict = hit.to_dict()
data_dict

{'dataset_type': 'L3_DSWx_HLS',
 'metadata': {'lineage': ['/data/work/jobs/2023/02/12/00/05/SCIFLO_L3_DSWx_HLS__1.0.0-rc.8.0-HLS.L30.T41TPF.2022340T061737.v2.0-20230210T192934.184974Z/HLS.L30.T41TPF.2022340T061737.v2.0.B02.tif',
   '/data/work/jobs/2023/02/12/00/05/SCIFLO_L3_DSWx_HLS__1.0.0-rc.8.0-HLS.L30.T41TPF.2022340T061737.v2.0-20230210T192934.184974Z/HLS.L30.T41TPF.2022340T061737.v2.0.B05.tif',
   '/data/work/jobs/2023/02/12/00/05/SCIFLO_L3_DSWx_HLS__1.0.0-rc.8.0-HLS.L30.T41TPF.2022340T061737.v2.0-20230210T192934.184974Z/HLS.L30.T41TPF.2022340T061737.v2.0.Fmask.tif',
   '/data/work/jobs/2023/02/12/00/05/SCIFLO_L3_DSWx_HLS__1.0.0-rc.8.0-HLS.L30.T41TPF.2022340T061737.v2.0-20230210T192934.184974Z/HLS.L30.T41TPF.2022340T061737.v2.0.B06.tif',
   '/data/work/jobs/2023/02/12/00/05/SCIFLO_L3_DSWx_HLS__1.0.0-rc.8.0-HLS.L30.T41TPF.2022340T061737.v2.0-20230210T192934.184974Z/HLS.L30.T41TPF.2022340T061737.v2.0.B03.tif',
   '/data/work/jobs/2023/02/12/00/05/SCIFLO_L3_DSWx_HLS__1.0.0-rc.8.0-HLS

# Get SAS from Product

In [15]:
import rasterio

In [21]:
with rasterio.open(data_dict['metadata']['product_urls'][0]) as ds:
    t = ds.tags()

In [23]:
t['SOFTWARE_VERSION']

'0.5.2'